In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)

learning_rate = 0.001
total_epoch = 30
batch_size = 128

n_input = 28
n_step = 28
n_hidden =128
n_class =10

# 2차원의 입력값을 가로픽셀수를 n_input(한번에 몇개?) 으로 , n_step(입력단계) 이라는 차원을 추가
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# 긴데이터를 학습할 시에 앞쪽을 잘 기억못하는 경향이 있음 -> LSTM or GRU를 씀
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
# dynamic_rnn 함수를 이용해 RNN 신경망을 완성
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype= tf.float32)
# 위 dynamic_rnn 이 아래 코드 대체
# states = tf.zeros(batch_size)
# for i in range(n_step):
#     output, states = cell(X[[:, i]], states)

# 은닉층의 출력값을 가중치 W와 같은 형태로 만들어줘야 행렬곱을 수행하여 원하는 출력값을 얻을 수 있음
# transpose함수를 이용하여 n_step 과 batch_size 차원의 순서를 바꿈, n_step차원을 제거하여 마지막 단계의 결과값만 취함
# 원래 [batch_size, n_step, n_hidden] 인것을 transpose 를 통해
# [n_step, batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])
# 그리고 [batch_size, n_hidden] 만 잘라 저장
outputs = outputs[-1]

# y = x * W +b
model = tf.matmul(outputs, W) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels =Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(total_epoch):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 입력값이 [batch_size, n_step, n_input] 형태이므로 reshape 함수를 이용해 데이터 형태를 바꿔줌
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost], feed_dict = {X:batch_xs, Y: batch_ys})
        total_cost += cost_val
    
    print('Epoch:', '%04d' % (epoch +1), 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!')
                      
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy, feed_dict={X:test_xs, Y:test_ys}))


Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch: 0001 Avg. cost = 0.510
Epoch: 0002 Avg. cost = 0.234
Epoch: 0003 Avg. cost = 0.182
Epoch: 0004 Avg. cost = 0.151
Epoch: 0005 Avg. cost = 0.134
Epoch: 0006 Avg. cost = 0.121
Epoch: 0007 Avg. cost = 0.114
